In [6]:
import sqlite3
import pandas as pd
import numpy as np

In [101]:
#need to find a way so this isn't called every time the page is run, use g global variable
#do some data cleaning: remove profiles that list 

def make_table():
    with sqlite3.connect("voter_data.db") as conn:
        df = pd.read_csv("~/Documents/GitHub/course-project-16b/alaska_presidentialelection.csv")
        df = df[["rank1","rank2","rank3","rank4","rank5"]]
        df.to_sql("votes",conn,index=False,if_exists="replace")
        
        cursor = conn.cursor()
        for i in range(1,6):
            cmd = f"""
            UPDATE votes 
            SET rank{i} = REPLACE(REPLACE(REPLACE(rank{i},"overvote","N/A"),"skipped","N/A"),"Undeclared","N/A")
            """
            cursor.execute(cmd)
            conn.commit()

        

In [102]:
make_table()

In [103]:
with sqlite3.connect("voter_data.db") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM votes LIMIT 5")
    print(cursor.fetchall())
    

[('Joseph R. Biden', 'Amy Klobuchar', 'Pete Buttigieg', 'Tom Steyer', 'Elizabeth Warren'), ('Joseph R. Biden', 'N/A', 'N/A', 'N/A', 'N/A'), ('Joseph R. Biden', 'N/A', 'N/A', 'N/A', 'N/A'), ('Bernie Sanders', 'Joseph R. Biden', 'N/A', 'N/A', 'N/A'), ('Joseph R. Biden', 'Elizabeth Warren', 'N/A', 'N/A', 'N/A')]


In [104]:
with sqlite3.connect("voter_data.db") as conn:
    cursor = conn.cursor()
    results = []
    for col in ["rank1","rank2","rank3","rank4","rank5"]:
        cmd = f"SELECT DISTINCT {col} FROM votes"
        cursor.execute(cmd)
        results += list(zip(*cursor.fetchall()))[0]
        
results = list(set(results))
results
#clean later to remove overvote, skipped, undeclared

['Tom Steyer',
 'Tulsi Gabbard',
 'Bernie Sanders',
 'Michael R. Bloomberg',
 'N/A',
 'Elizabeth Warren',
 'Joseph R. Biden',
 'Pete Buttigieg',
 'Amy Klobuchar']

In [39]:
results = ['Tulsi Gabbard',
 'Elizabeth Warren',
 'Pete Buttigieg',
 'Michael R. Bloomberg',
 'Tom Steyer',
 'Joseph R. Biden',
 'Bernie Sanders',
 'Amy Klobuchar']

In [85]:
def get_ranks(name,db_name):
    with sqlite3.connect("voter_data.db") as conn:
        cursor = conn.cursor()
        cmd = f"""
        SELECT CASE
            WHEN '{name}' IN (rank1) THEN 1
            WHEN '{name}' IN (rank2) THEN 2
            WHEN '{name}' IN (rank3) THEN 3
            WHEN '{name}' IN (rank4) THEN 4
            WHEN '{name}' IN (rank5) THEN 5
            ELSE 6
        END FROM {db_name}
        """ #if ranked multiple times, returns highest rank
        cursor.execute(cmd)
        return np.array(list(zip(*cursor.fetchall()))[0])
    
#get_ranks("Tulsi Gabbard")

In [76]:
def plurality(db_name):
    with sqlite3.connect("voter_data.db") as conn:
        cursor = conn.cursor()
        cmd = f"""
        SELECT rank1, COUNT(rank1)
        FROM {db_name}
        GROUP BY rank1
        ORDER BY COUNT(rank1) DESC
        """
        cursor.execute(cmd)
        return cursor.fetchall()

In [78]:
plurality("votes")

[('Joseph R. Biden', 9860),
 ('Bernie Sanders', 7762),
 ('Elizabeth Warren', 1399),
 ('Pete Buttigieg', 247),
 ('Amy Klobuchar', 131),
 ('Tulsi Gabbard', 124),
 ('Michael R. Bloomberg', 97),
 ('Undeclared', 64),
 ('Tom Steyer', 64),
 ('overvote', 32),
 ('skipped', 31)]

In [81]:
def borda(db_name,point_dict={1:1,2:2,3:3,4:4,5:5}):
    #ask if there's a more efficient way to do this
    count = {}
    with sqlite3.connect("voter_data.db") as conn:
        cursor = conn.cursor()
        for result in results:
            for col in [1,2,3,4,5]:
                cmd = f"SELECT {point_dict[col]}*COUNT(rank{col}) FROM {db_name} WHERE (rank{col}=='{result}')"
                cursor.execute(cmd)
                count[result] = count.get(result,0) + cursor.fetchone()[0]
    return [(i,j) for i,j in count.items()]

In [84]:
borda("votes")

[('Tulsi Gabbard', 6817),
 ('Elizabeth Warren', 23377),
 ('Pete Buttigieg', 19055),
 ('Michael R. Bloomberg', 8535),
 ('Tom Steyer', 7766),
 ('Joseph R. Biden', 29815),
 ('Bernie Sanders', 28724),
 ('Amy Klobuchar', 19112)]

In [83]:
def preference_profiles():
    with sqlite3.connect("voter_data.db") as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT *,COUNT(*) FROM votes \
        GROUP BY rank1,rank2,rank3,rank4,rank5 \
        ORDER BY COUNT(*) DESC")
    return cursor.fetchall()
 
len(preference_profiles())

2467

In [44]:
def IRV(db_name):
    #brute-force implementation
    #output is not necessarily a good ranking of preference, since worst candidates might
    #get no first place votes but more second-place votes
    #doesn't handle N/A votes well
    can_win = list(results)
    to_return = []
    with sqlite3.connect("voter_data.db") as conn:
        count = 0
        cursor = conn.cursor()
        while len(can_win)>1:
            votes=[]
            tallies=[]
            can_win_str = tuple(can_win)
            
            cmd = f"SELECT rank1,COUNT(rank1) FROM {db_name} WHERE rank1 IN {can_win_str} GROUP BY rank1"
            cursor.execute(cmd)
            votes += [cursor.fetchall()]
            
            cmd = f"SELECT rank2,COUNT(rank2) FROM {db_name} \
            WHERE rank2 IN {can_win_str} AND rank1 NOT IN {can_win_str} GROUP BY rank2"
            cursor.execute(cmd)
            votes += [cursor.fetchall()]
            
            cmd = f"SELECT rank3,COUNT(rank3) FROM {db_name} \
            WHERE rank3 IN {can_win_str} AND rank1 NOT IN {can_win_str} AND rank2 NOT IN {can_win_str} \
            GROUP BY rank3"
            cursor.execute(cmd)
            votes += [cursor.fetchall()]
            
            cmd = f"SELECT rank4,COUNT(rank4) FROM {db_name} \
            WHERE rank4 IN {can_win_str} AND rank1 NOT IN {can_win_str} AND rank2 NOT IN {can_win_str} \
            AND rank3 NOT IN {can_win_str} \
            GROUP BY rank4"
            cursor.execute(cmd)
            votes += [cursor.fetchall()]
            
            cmd = f"SELECT rank5,COUNT(rank5) FROM {db_name} \
            WHERE rank5 IN {can_win_str} AND rank1 NOT IN {can_win_str} AND rank2 NOT IN {can_win_str} \
            AND rank3 NOT IN {can_win_str} AND rank4 NOT IN {can_win_str} \
            GROUP BY rank5"
            cursor.execute(cmd)
            votes += [cursor.fetchall()]
            
            for person in can_win:
                vote_count = sum([dict(votes[i]).get(person,0) for i in range(5)])
                tallies += [(person,vote_count)]
                
            tallies = sorted(tallies,key=lambda x:-x[1])
            print(tallies)
            last_place = tallies[-1][0]
            
            to_return = [last_place] + to_return
            can_win.remove(last_place)
            
        return can_win + to_return

In [45]:
IRV("votes")

[('Joseph R. Biden', 9875), ('Bernie Sanders', 7771), ('Elizabeth Warren', 1411), ('Pete Buttigieg', 250), ('Amy Klobuchar', 138), ('Tulsi Gabbard', 131), ('Michael R. Bloomberg', 99), ('Tom Steyer', 67)]
[('Joseph R. Biden', 9894), ('Bernie Sanders', 7785), ('Elizabeth Warren', 1420), ('Pete Buttigieg', 257), ('Amy Klobuchar', 144), ('Tulsi Gabbard', 133), ('Michael R. Bloomberg', 107)]
[('Joseph R. Biden', 9948), ('Bernie Sanders', 7801), ('Elizabeth Warren', 1430), ('Pete Buttigieg', 268), ('Amy Klobuchar', 150), ('Tulsi Gabbard', 134)]
[('Joseph R. Biden', 9970), ('Bernie Sanders', 7851), ('Elizabeth Warren', 1440), ('Pete Buttigieg', 281), ('Amy Klobuchar', 167)]
[('Joseph R. Biden', 10013), ('Bernie Sanders', 7867), ('Elizabeth Warren', 1495), ('Pete Buttigieg', 327)]
[('Joseph R. Biden', 10151), ('Bernie Sanders', 7923), ('Elizabeth Warren', 1610)]
[('Joseph R. Biden', 10839), ('Bernie Sanders', 8761)]


['Joseph R. Biden',
 'Bernie Sanders',
 'Elizabeth Warren',
 'Pete Buttigieg',
 'Amy Klobuchar',
 'Tulsi Gabbard',
 'Michael R. Bloomberg',
 'Tom Steyer']

In [89]:
def TopTwo(db_name):
    plural = plurality(db_name)
    top_two = [plural[0][0],plural[1][0]]
    count1 = np.sum([get_ranks(top_two[0],db_name)<get_ranks(top_two[1],db_name)])
    count2 = np.sum([get_ranks(top_two[0],db_name)>get_ranks(top_two[1],db_name)])
    results = list(zip(top_two,[count1,count2]))
    if count1 > count2:
        return results
    else:
        return results.reverse()

In [90]:
TopTwo("votes")

[('Joseph R. Biden', 10839), ('Bernie Sanders', 8761)]